## A CLASSIFICATION PROJECT - THE SEPSIS CASE STUDY

#### BUSINESS UNDERSTANDING
Sepsis is a critical health condition with significant implications for patient outcomes and healthcare systems. Predicting and managing sepsis effectively can reduce mortality rates, improve 

patient recovery, and decrease healthcare costs. This project aims to leverage machine learning models and FastAPI to predict sepsis, thereby enabling timely interventions and enhancing clinical 

decision-making. This project aims to enhance the early detection and management of sepsis through a machine learning-based predictive model deployed as an API with FastAPI. By addressing a 

critical healthcare challenge, the project seeks to improve patient outcomes, optimize resource utilization, and provide valuable decision support for healthcare providers. The success of the 

project will be measured by its impact on clinical practice and its ability to deliver timely, accurate predictions in a real-world healthcare setting.

#### Business Objectives
1. Early Detection: Develop a predictive model to identify patients at risk of sepsis early, allowing for prompt intervention and treatment.

2. Reduce Mortality Rates: Use the predictive model to minimize the time to diagnosis and treatment, thereby reducing sepsis-related deaths.

3. Optimize Resource Utilization: Allocate medical resources more efficiently by identifying high-risk patients, reducing unnecessary testing and treatments for low-risk individuals.

4. Enhance Clinical Decision-Making: Provide healthcare professionals with reliable tools to support clinical decisions, improving patient care quality.

##### HYPOTHESIS
NULL HYPOTHESIS: There is no significant relationship between sepsis and PRG (Plasma/glucose).

ALTERNATE HYPOTHESIS: There is a significant relationship between sepsis and PRG (Plasma /glucose).

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.
